Nicholas Galea - 409904L

CIS3190 - Data Analytics Assignment

# Stage 1 - EDA Exploratory Data Analysis

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv("telecom_customer_churn.csv")

# Replace ' ' with NaN and drop rows where TotalCharges is NaN
data['TotalCharges'] = data['TotalCharges'].replace(' ', np.nan)  # Replace blank spaces with NaN
data = data.dropna(subset=['TotalCharges'])  # Drop rows with NaN in TotalCharges

# Print column names to debug
for col in data.columns:
    print('Column:', col, '\t', data[col].dtype)
    print(data[col].unique())
    print()
    


Column: customerID 	 object
['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']

Column: gender 	 object
['Female' 'Male']

Column: SeniorCitizen 	 int64
[0 1]

Column: Partner 	 object
['Yes' 'No']

Column: Dependents 	 object
['No' 'Yes']

Column: tenure 	 int64
[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]

Column: PhoneService 	 object
['No' 'Yes']

Column: MultipleLines 	 object
['No phone service' 'No' 'Yes']

Column: InternetService 	 object
['DSL' 'Fiber optic' 'No']

Column: OnlineSecurity 	 object
['No' 'Yes' 'No internet service']

Column: OnlineBackup 	 object
['Yes' 'No' 'No internet service']

Column: DeviceProtection 	 object
['No' 'Yes' 'No internet service']

Column: TechSupport 	 object
['No' 'Yes' 'No internet service']

Column: StreamingTV 	 object
['No' 'Yes

In [ ]:
categorical_columns = ['InternetService', 'OnlineSecurity', 'MultipleLines',
                       'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                       'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']

yes_no_columns = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
two_value_columns = ['gender']

columns = list(set(categorical_columns).union(yes_no_columns, two_value_columns))

print(columns)

# SeniorCitizen is already 0/1.

In [ ]:
# Display basic information about the dataset
print("Data Information")
data.info()

# Summary Statistics for Numerical Columns
print("Statistcs for Numerical Columns")
data.describe()

# Confirming that no 'nan' values are present
print("nan values")
data.isnull().sum()

In [ ]:
# Plot the distribution of the target variable
plt.figure(figsize=(8, 6))
sns.countplot(data=data, x='Churn', palette='viridis')
plt.title('Distribution of Churn')
plt.xlabel('Churn')
plt.ylabel('Count')
plt.show()

In [ ]:
# Plot the distribution of categorical features
for column in columns:
    plt.figure(figsize=(8, 6))
    sns.countplot(data=data, x=column, palette='viridis')
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# List of numerical columns
numerical_columns = ['tenure', 'MonthlyCharges', 'TotalCharges']

# Plot the distribution of numerical features
for column in numerical_columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(data=data, x=column, kde=True, bins=30, color='blue')
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
# Plot churn rates by categorical features
for column in categorical_columns:
    plt.figure(figsize=(8, 6))
    sns.barplot(data=data, x=column, y=data['Churn'].map({'Yes': 1, 'No': 0}), palette='viridis')
    plt.title(f'Churn Rate by {column}')
    plt.xlabel(column)
    plt.ylabel('Churn Rate')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Convert Churn column from Yes/No to Binary 0/1
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

In [ ]:
# Convert Churn to numerical for correlation
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

# Calculate correlation matrix
correlation_matrix = data[numerical_columns + ['Churn']].corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# One-Hot-Encoding
print(data.columns)
print(data.head())

In [ ]:
# Check the columns list
print("Columns list:")
print(columns)

# Compare with DataFrame columns
print("DataFrame columns:")
print(data.columns)

# Check if all `columns` are in `data`
missing_columns = [col for col in columns if col not in data.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    print("All columns are present in the DataFrame.")

In [ ]:
# Dropping CustomerID (Not Needed)
if 'customerID' in data.columns:
    data = data.drop(columns=['customerID'])

# Doing One-Hot-Encoding 

# 1. Gender Column: Male -> 0, Female -> 1
data['gender'] = data['gender'].map({'Male': 0, 'Female': 1})
print("(Gender) Data Frame columns:", data.columns)

# 2. Converting Yes -> 1, No -> 0
for col in yes_no_columns:
    data[col] = data[col].map({'Yes': 1, 'No': 0})
print("Yes/No Data Frame columns:", data.columns)

# 3. One-Hot-Encoding Categorical Columns
# Function to generate one-hot mapping dynamically
def generate_one_hot_mapping(unique_values):
    mapping = {value: [1 if i == j else 0 for i in range(len(unique_values))]
        for j, value in enumerate(unique_values)}
    return mapping

for col in categorical_columns:
    # Generate the mapping dynamically
    
    unique_values = data[col].unique()  # Get unique values from the column
    one_hot_mapping = generate_one_hot_mapping(unique_values)
    
    # Replace string values with one-hot-encoded lists
    data[col] = data[col].map(one_hot_mapping)

# 4. Ensure all columns are numeric
# Convert all Boolean columns: True -> 1, False -> 0
data = data.applymap(lambda x: 1 if x is True else (0 if x is False else x))

# 5. Verify that all columns are numeric
# assert all(data.dtypes.apply(lambda dtype: np.issubdtype(dtype, np.number))), "Not all columns are numeric!"

# Check the results
print(data.head())

In [ ]:
# Non-Numeric Columns
non_numeric_columns = data.select_dtypes(exclude=['number']).columns

print("Non-numeric columns:")
print(non_numeric_columns)

print(data.head(10))

In [ ]:
df_X = data.drop(columns=['Churn'])
df_y = data['Churn']

# Convert dataframces to Numpy arrrays
X = df_X.to_numpy()
y = df_y.to_numpy()

print(data.head())
print()
print(df_X.head())
print()
print(df_y.head())
print()
print(X)
print()
print(y)

In [ ]:
# Split the data into training(80%) and testing(20%) sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of training and testing sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

In [ ]:
# Calculating Basic Statistics
    
print("Basic Statistics:")
print(data.describe()) 

print("\nMode for each column:")
print(data.mode().iloc[0])  # Get mode for each column

In [ ]:
# Distribution of features

import matplotlib.pyplot as plt
import seaborn as sns

# Set the plot style to be more aesthetic
sns.set(style="whitegrid")

# 1. Distribution of Numerical Features
numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges']

plt.figure(figsize=(15, 5))
for i, feature in enumerate(numerical_features, 1):
    plt.subplot(1, 3, i)
    sns.histplot(data[feature], kde=True, bins=30, color="skyblue")
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# 2. Distribution of Categorical Features
categorical_features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 
                        'PhoneService', 'PaperlessBilling', 'Churn']

plt.figure(figsize=(15, 10))
for i, feature in enumerate(categorical_features, 1):
    plt.subplot(3, 3, i)
    sns.countplot(data[feature], palette="pastel")
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Count')

plt.tight_layout()
plt.show()
